In [1]:
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import signal, stats
import mat73
import re
from neurodsp.timefrequency import compute_wavelet_transform
import os
import mne
import IPython
import seaborn as sns
import scipy
import joblib
import h5io
import dask.array as da 
import itertools

import statsmodels
from statsmodels import stats
from statsmodels.stats import multitest

# Import required code for visualizing example models
from fooof import FOOOF
from fooof.sim.gen import gen_power_spectrum
from fooof.sim.utils import set_random_seed
from fooof.plts.spectra import plot_spectra
from fooof.plts.annotate import plot_annotated_model
from neurodsp.utils import create_times
from neurodsp.plts.time_series import plot_time_series
from neurodsp.spectral import compute_spectrum, rotate_powerlaw
from neurodsp.plts.spectral import plot_power_spectra


In [2]:

# Subject and ROI lists
sub_list = ['BJH021', 'BJH025', 'BJH016', 'SLCH002', 'BJH026', 'BJH027', 'BJH029', 'BJH039', 'BJH041', 'LL10', 'LL12', 'LL13', 'LL14', 'LL17', 'LL19']

# Prepare DataFrame
sub_roi_df = pd.DataFrame({'subject': sub_list})

# Display the DataFrame
print(sub_roi_df)


    subject
0    BJH021
1    BJH025
2    BJH016
3   SLCH002
4    BJH026
5    BJH027
6    BJH029
7    BJH039
8    BJH041
9      LL10
10     LL12
11     LL13
12     LL14
13     LL17
14     LL19


In [3]:
for sub in sub_list:
    # load in raw data to get good epochs #
    preproc_data_dir = f"/home/brooke/pacman/preprocessing/{sub}/ieeg"
    first_move_epochs = mne.read_epochs(f"{preproc_data_dir}/{sub}_bp_filtered_clean_first_move_events.fif")

    ## Dictionary of electrode locations ##
    # Pull mapping ROI to elecs
    %run /home/brooke/pacman/preprocessing/scripts/roi.py
    ROIs = ROIs[sub]

    ## prep lists

    # limbic ROI
    hc_list = []
    ofc_list = []
    amyg_list = []
    cing_list = []
    sgACC_list = []
    dACC_list = []
    dlpfc_list = []
    sfg_list = []
    mfg_list = []

    # control ROI
    insula_list = []
    dlpfc_list = []
    ec_list = []
    
    # exclude bad ROI from list
    channels = [item for item in first_move_epochs.info['ch_names'] if '-' in item]
    pairs_long_name = [ch.split('-') for ch in channels]
    bidx = len(first_move_epochs.info['bads']) +1
    pairs_name = pairs_long_name[bidx:len(pairs_long_name)]


    # sort ROI into lists
    for ix in range(0, len(pairs_name)):
        if pairs_name[ix][0] in ROIs['hc'] or pairs_name[ix][1] in ROIs['hc']:
            hc_list.append(first_move_epochs.info['ch_names'][ix + bidx])
        if pairs_name[ix][0] in ROIs['ofc'] or pairs_name[ix][1] in ROIs['ofc']:
            ofc_list.append(first_move_epochs.info['ch_names'][ix + bidx])
        if pairs_name[ix][0] in ROIs['amyg'] or pairs_name[ix][1] in ROIs['amyg']:
            amyg_list.append(first_move_epochs.info['ch_names'][ix + bidx])       
        if pairs_name[ix][0] in ROIs['cing'] or pairs_name[ix][1] in ROIs['cing']:
            cing_list.append(first_move_epochs.info['ch_names'][ix + bidx])       
        if pairs_name[ix][0] in ROIs['sgACC'] or pairs_name[ix][1] in ROIs['sgACC']:
            sgACC_list.append(first_move_epochs.info['ch_names'][ix + bidx])       
        if pairs_name[ix][0] in ROIs['dACC'] or pairs_name[ix][1] in ROIs['dACC']:
            dACC_list.append(first_move_epochs.info['ch_names'][ix + bidx])            

        # control roi
        if pairs_name[ix][0] in ROIs['insula'] or pairs_name[ix][1] in ROIs['insula']:
            insula_list.append(first_move_epochs.info['ch_names'][ix + bidx])       
        if pairs_name[ix][0] in ROIs['dlpfc'] or pairs_name[ix][1] in ROIs['dlpfc']:
            dlpfc_list.append(first_move_epochs.info['ch_names'][ix + bidx])       
        if pairs_name[ix][0] in ROIs['sfg'] or pairs_name[ix][1] in ROIs['sfg']:
            sfg_list.append(first_move_epochs.info['ch_names'][ix + bidx])       
        if pairs_name[ix][0] in ROIs['mfg'] or pairs_name[ix][1] in ROIs['mfg']:
            mfg_list.append(first_move_epochs.info['ch_names'][ix + bidx])       
        if pairs_name[ix][0] in ROIs['ec'] or pairs_name[ix][1] in ROIs['ec']:
            ec_list.append(first_move_epochs.info['ch_names'][ix + bidx])   
            
            
    ## add to sub_roi_df
    sub_roi_df.loc[sub_roi_df['subject'] == sub, 'hc'] = len(hc_list)
    sub_roi_df.loc[sub_roi_df['subject'] == sub, 'amyg'] = len(amyg_list)
    sub_roi_df.loc[sub_roi_df['subject'] == sub, 'ofc'] = len(ofc_list)
    sub_roi_df.loc[sub_roi_df['subject'] == sub, 'cing'] = len(cing_list)
    sub_roi_df.loc[sub_roi_df['subject'] == sub, 'sgACC'] = len(sgACC_list)
    sub_roi_df.loc[sub_roi_df['subject'] == sub, 'dACC'] = len(dACC_list)
    sub_roi_df.loc[sub_roi_df['subject'] == sub, 'sfg'] = len(sfg_list)
    sub_roi_df.loc[sub_roi_df['subject'] == sub, 'mfg'] = len(mfg_list)
    sub_roi_df.loc[sub_roi_df['subject'] == sub, 'insula'] = len(insula_list)
    sub_roi_df.loc[sub_roi_df['subject'] == sub, 'dlpfc'] = len(dlpfc_list)
    
           

Reading /home/brooke/pacman/preprocessing/BJH021/ieeg/BJH021_bp_filtered_clean_first_move_events.fif ...


/tmp/ipykernel_5360/541109368.py:4: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH021/ieeg/BJH021_bp_filtered_clean_first_move_events.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  first_move_epochs = mne.read_epochs(f"{preproc_data_dir}/{sub}_bp_filtered_clean_first_move_events.fif")


    Found the data of interest:
        t =   -5000.00 ...    5000.00 ms
        0 CTF compensation matrices available
Reading /home/brooke/pacman/preprocessing/BJH021/ieeg/BJH021_bp_filtered_clean_first_move_events-1.fif ...
    Found the data of interest:
        t =   -5000.00 ...    5000.00 ms
        0 CTF compensation matrices available
Not setting metadata
239 matching events found
No baseline correction applied
0 projection items activated
Reading /home/brooke/pacman/preprocessing/BJH025/ieeg/BJH025_bp_filtered_clean_first_move_events.fif ...


/tmp/ipykernel_5360/541109368.py:4: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH025/ieeg/BJH025_bp_filtered_clean_first_move_events.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  first_move_epochs = mne.read_epochs(f"{preproc_data_dir}/{sub}_bp_filtered_clean_first_move_events.fif")


    Found the data of interest:
        t =   -2000.00 ...    5000.00 ms
        0 CTF compensation matrices available
Reading /home/brooke/pacman/preprocessing/BJH025/ieeg/BJH025_bp_filtered_clean_first_move_events-1.fif ...
    Found the data of interest:
        t =   -2000.00 ...    5000.00 ms
        0 CTF compensation matrices available
Not setting metadata
238 matching events found
No baseline correction applied
0 projection items activated
Reading /home/brooke/pacman/preprocessing/BJH016/ieeg/BJH016_bp_filtered_clean_first_move_events.fif ...


...

    Found the data of interest:
        t =   -5000.00 ...    5000.00 ms
        0 CTF compensation matrices available
Not setting metadata
815 matching events found
No baseline correction applied
0 projection items activated
Reading /home/brooke/pacman/preprocessing/LL17/ieeg/LL17_bp_filtered_clean_first_move_events.fif ...
    Found the data of interest:
        t =   -5000.00 ...    5000.00 ms
        0 CTF compensation matrices available


/tmp/ipykernel_5360/541109368.py:4: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/LL17/ieeg/LL17_bp_filtered_clean_first_move_events.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  first_move_epochs = mne.read_epochs(f"{preproc_data_dir}/{sub}_bp_filtered_clean_first_move_events.fif")


Not setting metadata
245 matching events found
No baseline correction applied
0 projection items activated
Reading /home/brooke/pacman/preprocessing/LL19/ieeg/LL19_bp_filtered_clean_first_move_events.fif ...
    Found the data of interest:
        t =   -5000.00 ...    5000.00 ms
        0 CTF compensation matrices available


/tmp/ipykernel_5360/541109368.py:4: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/LL19/ieeg/LL19_bp_filtered_clean_first_move_events.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  first_move_epochs = mne.read_epochs(f"{preproc_data_dir}/{sub}_bp_filtered_clean_first_move_events.fif")


Not setting metadata
238 matching events found
No baseline correction applied
0 projection items activated


In [4]:
sub_roi_df

,subject,hc,amyg,ofc,cing,sgACC,dACC,sfg,mfg,insula,dlpfc
0,BJH021,5.0,8.0,10.0,4.0,0.0,4.0,0.0,9.0,14.0,9.0
1,BJH025,6.0,12.0,12.0,2.0,2.0,0.0,4.0,4.0,12.0,8.0
2,BJH016,9.0,3.0,14.0,8.0,4.0,4.0,0.0,12.0,12.0,12.0
3,SLCH002,10.0,3.0,13.0,7.0,7.0,0.0,0.0,6.0,14.0,6.0
4,BJH026,0.0,4.0,2.0,2.0,2.0,0.0,0.0,5.0,2.0,5.0
5,BJH027,6.0,11.0,12.0,0.0,0.0,0.0,0.0,4.0,16.0,4.0
6,BJH029,13.0,10.0,7.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0
7,BJH039,13.0,1.0,18.0,9.0,1.0,8.0,0.0,20.0,0.0,20.0
8,BJH041,9.0,5.0,5.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0
9,LL10,4.0,1.0,1.0,6.0,1.0,5.0,0.0,10.0,0.0,10.0


In [7]:

sum_of_counts = sub_roi_df.drop('subject', axis=1).sum()
print(sum_of_counts)


hc         87.0
amyg       66.0
ofc       104.0
cing       45.0
sgACC      20.0
dACC       25.0
sfg        20.0
mfg       117.0
insula     90.0
dlpfc     134.0
dtype: float64


In [9]:
first_subs = ['BJH021', 'BJH025', 'BJH016', 'SLCH002', 'LL10', 'LL12', 'LL13']
first_cohort_df = sub_roi_df[sub_roi_df['subject'].isin(first_subs)]
first_cohort_df.drop('subject', axis=1).sum()

hc        37.0
amyg      31.0
ofc       53.0
cing      31.0
sgACC     14.0
dACC      17.0
sfg       15.0
mfg       58.0
insula    55.0
dlpfc     72.0
dtype: float64

In [10]:
first_subs = ['BJH021', 'BJH025', 'BJH016', 'SLCH002', 'LL10', 'LL12', 'LL13']
first_cohort_df = sub_roi_df[~sub_roi_df['subject'].isin(first_subs)]
first_cohort_df.drop('subject', axis=1).sum()

hc        50.0
amyg      35.0
ofc       51.0
cing      14.0
sgACC      6.0
dACC       8.0
sfg        5.0
mfg       59.0
insula    35.0
dlpfc     62.0
dtype: float64

In [11]:
# Total subjects

len(sub_list)

15